In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
from datetime import datetime
import base64
import hashlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from typing import List, Dict
from typing import Callable

# Caminho para o arquivo xlsx local
file_path = './banco_dados.xlsx'

In [39]:
# Funções de conversão
def substituir_string(texto, lista_substituicoes):
    for antigo, novo in lista_substituicoes:
        texto = texto.replace(antigo, novo)
    return texto

def string_to_unique_float(s: str) -> float:
    # Gera um hash MD5 da string
    md5_hash = hashlib.md5(s.encode()).hexdigest()
    # Converte o hash em um número inteiro
    int_value = int(md5_hash, 16)
    # Normaliza o valor para o intervalo entre [0,1] e retorna
    return int_value % (10**8) / (10**8)

def replace_with_other(dataResp: Dict[str, list[str]],colNum: int, text,list: List,func: Callable[[str], str] = string_to_unique_float):
        text = substituir_string(text,list)
        if text.startswith("other:") :
            appendData(dataResp, f'Resp_{colNum}', func(text.replace("other:","")))
        else:
            appendData(dataResp, f'Resp_{colNum}', float(text))             

def appendData(dataResp: Dict[str, list[str]], key: str, colData: str):
    try:
        dataResp[key].append(colData)
    except KeyError:
        dataResp[key] = [colData]


def trataCol(dataResp: Dict[str, list[str]], colNum: int, colData: str, timeAtual: pd.Timestamp, timeAnterior: pd.Timestamp):
    timeElipsed = (timeAtual - timeAnterior).total_seconds()    
    colData = colData.replace("other;", "other:").replace("Sucess;", "").replace("Sucess", "")
    match colNum:
        case 2:
            for index,item in enumerate(colData.split(";")):
                match index:
                    case 0: # Que horas são neste instante?
                        hora = item
                    case 1: # Data de hoje?
                        # Calcula o quanto o entrevistado estava errado em noção do tempo
                        appendData(dataResp, 'Resp_1', (timeAtual - pd.to_datetime(f"{item} {hora}", dayfirst=True)).total_seconds())
                    case 2: # Qual a sua Idade?
                        appendData(dataResp, 'Resp_2', item)                       
                    case 3: # Gênero *
                        replace_with_other(dataResp,index,item,[("Feminino", "0.0"), ("Masculino", "1.0")])                         
                    case 4: # Qual foi o sexo atribuído no seu nascimento?
                        replace_with_other(dataResp,index,item,[("Feminino", "0.0"), ("Masculino", "1.0")])
                    case 5: # Assinale a alternativa que identifica a sua Cor ou Raça:
                        appendData(dataResp, f'Resp_{index}', string_to_unique_float(item.replace("other:","")))                        
                        # replace_with_other(dataResp,index,item,[("Preta", "0"), ("Branca", "1"), ("Parda", "2"), ("Amarela", "3"), ("IndÍgena", "4")])
                    case 6: # Dentro de sua família, você é o(a) único(a) filho(a)?
                        replace_with_other(dataResp,index,item,[("Sim", "0.0")],lambda x: float(x))
                    case 7: # Qual o seu estado civil?
                        appendData(dataResp, f'Resp_{index}', string_to_unique_float(item.replace("other:","")))   
                    case 8: # Possui filhos(as)?
                        replace_with_other(dataResp,index,item,[("Não", "0.0")],lambda x: float(x)) 
                    case 9: # Possui filhos(as) menores de 6 anos?
                        replace_with_other(dataResp,index,item,[("Não", "0.0"), ("Sim", "1.0")])    
                    case 10: # Religião *
                        replace_with_other(dataResp,index,item,[("Sem religião", "0.0")])
                    case 11: # Escolaridade *
                        replace_with_other(dataResp,index,item,[
                            ("Sem Escolaridade", "0.0"),
                            ("Ensino Fundamental (1º grau) incompleto","0.1"),
                            ("Ensino Fundamental (1º grau) completo", "0.2"),
                            ("Ensino Médio (2º grau) incompleto", "0.3"),
                            ("Ensino Médio (2º grau) completo", "0.4"),
                            ("Superior Incompleto", "0.55"),
                            ("Superior Completo", "0.7"),
                            ("Mestrado", "0.85"),
                            ("Doutorado", "1.0")
                        ])
                    case 12: # Renda familiar mensal de sua casa (somatória)
                        replace_with_other(dataResp,index,item,[
                            ("Até 1 salário mínimo", "0.0"),
                            ("Mais de 1 a 2 salários mínimos", "0.15"),
                            ("Mais de 2 a 3 salários mínimos", "0.3"),
                            ("Mais de 3 a 5 salários mínimos", "0.45"),
                            ("Mais de 5 a 8 salários mínimos", "0.6"),
                            ("Mais de 8 a 12 salários mínimos", "0.75"),
                            ("Mais de 12 a 20 salários mínimos", "9.0"),
                            ("Mais de 20 salários mínimos", "1.0")
                        ])                     
                appendData(dataResp, f'Elipsed_{index}', timeElipsed)                            
        case 3:  
            replace_with_other(dataResp,13,colData,[(";", ""),("Não", "0.0")])
            appendData(dataResp, f'Elipsed_{13}', timeElipsed)  
        case 7:  
            replace_with_other(dataResp,14,colData,[
              ("Jesus Cristo", "0.0"),
              ("Coração", "0.05"),
              ("Dragão cuspindo fogo", "0.01"),
              ("Árvore", "1.0"),
              ("Não vi nada", "0.1"),
              ("Outra coisa", "0.15")
            ])           
            appendData(dataResp,'Elipsed_14', timeElipsed)               
        case 10:  
            replace_with_other(dataResp,15,colData,[
              ("Jesus Cristo", "0.0"),
              ("Coração", "0.05"),
              ("Dragão cuspindo fogo", "0.01"),
              ("Árvore", "0.1"),
              ("Não vi nada", "1.0"),
              ("Outra coisa", "0.15")
            ])           
            appendData(dataResp,'Elipsed_15', timeElipsed)
        case 13: # Qual foi a sequência correta dos números apresentados na tela anterior?
            replace_with_other(dataResp,16,colData,[
              ("1 - 5", "0.1"),
              ("4 - 7", "0.2"),
              ("2 - 7", "1.0"),
              ("2 - 8", "0.6"),
              ("9 - 4", "0.4"),
              ("7 - 2", "0.7")
            ])           
            appendData(dataResp, 'Elipsed_16', timeElipsed)
        case 15: # Qual foi a sequência correta dos números apresentados na tela anterior?
            replace_with_other(dataResp,17,colData,[
              ("5 - 7 - 1", "0.7"),
              ("1 - 3 - 4", "0.1"),
              ("5 - 6 - 3", "0.8"),
              ("4 - 6 - 5", "0.2"),
              ("5 - 4 - 6", "0.9"),
              ("5 - 6 - 4", "1.0")
            ])           
            appendData(dataResp, 'Elipsed_17', timeElipsed)
        case 17: # Qual foi a sequência correta dos números apresentados na tela anterior?
            replace_with_other(dataResp,18,colData,[
              ("6 - 4 - 9 - 3", "0.8"),
              ("4 - 8 - 9 - 1", "0.3"),
              ("6 - 4 - 3 - 9", "1.0"),
              ("5 - 4 - 3 - 8", "0.4"),
              ("1 - 5 - 2 - 9", "0.2"),
              ("6 - 4 - 3 - 7", "0.7")
            ])           
            appendData(dataResp, 'Elipsed_18', timeElipsed)
        case 19: # Qual foi a sequência correta dos números apresentados na tela anterior?
            replace_with_other(dataResp,19,colData,[
              ("2 - 1 - 4 - 7 - 9", "0.4"),
              ("4 - 3 - 9 - 8", "0.5"),
              ("4 - 2 - 6 - 3 - 1", "0.8"),
              ("7 - 5 - 1 - 4 - 2 ", "0.3"),
              ("4 - 2 - 7 - 3 - 1", "1.0"),
              ("6 - 3 - 1 - 5 - 9", "0.2")
            ])           
            appendData(dataResp, 'Elipsed_19', timeElipsed)
        case 21: # Qual foi a sequência correta dos números apresentados na tela anterior?
            replace_with_other(dataResp,20,colData,[
              ("6 - 1 - 4 - 7 - 3 - 9", "0.7"),
              ("2 - 1 - 8 - 3 - 9 - 5", "0.4"),
              ("6 - 1 - 9 - 4 - 7 - 3", "1.0"),
              ("6 - 4 - 5 - 8 - 3 - 7", "0.5"),
              ("2 - 8 - 6 - 4 - 7 - 3", "0.3"),
              ("6 - 1 - 9 - 4 - 5 - 2", "0.8")
            ])           
            appendData(dataResp, 'Elipsed_20', timeElipsed)
        case 23: # Qual foi a sequência correta dos números apresentados na tela anterior?
            replace_with_other(dataResp,21,colData,[
              ("5 - 8 - 1 - 7 - 4 - 3 - 8", "0.7"),
              ("5 - 9 - 1 - 7 - 4 - 8", "0.8"),
              ("8 - 9 - 0 - 7 - 3 - 1", "0.1"),
              ("5 - 9 - 1 - 7 - 4 - 2 - 8", "1.0"),
              ("5 - 2 - 3 - 7 - 4 - 9 - 8", "0.6"),
              ("5 - 9 - 1 - 7 - 8 - 0 - 9", "0.5")
            ])           
            appendData(dataResp, 'Elipsed_21', timeElipsed)
        case 25: # Dentre as quatro alternativas de cada imagem. Selecione a palavra que melhor a descreve
            for index,item in enumerate(colData.split(";")):
                match index:
                    case 0: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Inquieto(a)', "0.1"),
                            ('Pensativo(a)', "1.0"),
                            ('Irritado(a)', "0.2"),
                            ('Desconfiado(a)', "0.3")
                        ])
                    case 1: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Nervoso(a)', "0.1"),
                            ('Deprimido(a)', "0.2"),
                            ('Irritado(a)', "0.3"),
                            ('Divertido(a)', "1.0")                            
                        ])
                    case 2: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Envergonhado(a)', "0.1"),
                            ('Divertido(a)', "0.2"),
                            ('Interessado(a)', "1.0"),
                            ('Deprimido(a)', "0.3")                            
                        ])                      
                    case 3: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Arrogante', "0.3"),
                            ('Decidido(a)', "0.2"),
                            ('Apavorado(a)', "1.0"),
                            ('Chateado(a)', "0.1")                            
                        ])                        
                    case 4: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Amável', "0.1"),
                            ('Decidido(a)', "0.2"),
                            ('Simpático(a)', "0.3"),
                            ('Deprimido(a)', "1.0")                            
                        ])
                    case 5: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Tímido(a)', "0.1"),
                            ('Perturbado(a)', "1.0"),
                            ('Desanimado(a)', "0.2"),
                            ('Pensativo(a)', "0.3")                            
                        ])                        
                    case 6: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Impaciente', "1.0"),
                            ('Desanimado(a)', "0.2"),
                            ('Sedutor(a)', "0.1"),
                            ('Aliviado(a)', "0.3")                            
                        ])
                    case 7: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Grato(a)', "0.3"),
                            ('Sonhador(a)', "0.1"),
                            ('Desanimado(a)', "1.0"),
                            ('Chocado(a)', "0.2")                            
                        ])  
                    case 8: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Satisfeito(a)', "0.3"),
                            ('Preocupado(a)', "1.0"),
                            ('Afetuoso(a)', "0.1"),
                            ('Indiferente', "0.2")                            
                        ]) 
                    case 9: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Amável', "0.1"),
                            ('Arrependido(a)', "1.0"),
                            ('Zangado(a)', "0.2"),
                            ('Simpático(a)', "0.3")                            
                        ])   
                    case 10: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Desconfortável', "1.0"),
                            ('Entediado(a)', "0.1"),
                            ('Confiante', "0.2"),
                            ('Impaciente', "0.3")                            
                        ])
                    case 11: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Arrependido(a)', "0.2"),
                            ('Nervoso(a)', "1.0"),
                            ('Divertido(a)', "0.3"),
                            ('Envergonhado(a)', "0.1")                            
                        ])
                    case 12: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Amigável', "0.1"),
                            ('Entretido(a)', "0.2"),
                            ('Desconfiado(a)', "1.0"),
                            ('Sedutor(a)', "0.3")                            
                        ]) 
                    case 13: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Preocupado(a)', "0.2"),
                            ('Cauteloso(a)', "0.1"),
                            ('Hostil', "1.0"),
                            ('Divertido(a)', "0.3")                            
                        ])                                              
                appendData(dataResp, f'Elipsed_{22 + index}', timeElipsed) 
        case 26: # Responda as questões abaixo:
            colData = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData)              
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,36 + index,item,[
                    ('Não', "0.0"),
                    ('Sim', "1.0"),
                ])                                  
                appendData(dataResp, f'Elipsed_{36 + index}', timeElipsed) 
        case 27: # Qual das imagens abaixo completa a sequência a seguir?
            replace_with_other(dataResp,42,colData,[
              (";", ""),
              ("assets/intel_1", ""),
              ("a", "0.1"),
              ("b", "0.2"),
              ("c", "0.3"),
              ("d", "0.4"),
              ("e", "1.0"),
              ("f", "0.5")            
            ])           
            appendData(dataResp, 'Elipsed_42', timeElipsed)
        case 28: # Responda as questões abaixo:
            colData = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData)       
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,43 + index,item,[
                    ('Não', "0.0"),
                    ('Sim', "1.0"),
                ])                                    
                appendData(dataResp, f'Elipsed_{43 + index}', timeElipsed)
        case 29: # Complete as sequências a seguir:
            for index,item in enumerate(colData.split(";")):
                appendData(dataResp, f'Resp_{50 + index}', float(item))                                                                          
                appendData(dataResp, f'Elipsed_{50 + index}', timeElipsed)                                                                                                                                                                                        
        case _:
            appendData(dataResp, f'Resp_{24 + colNum}', re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData))
            appendData(dataResp, f'Elipsed_{24 + colNum}', timeElipsed)                
        

In [40]:
# Leitura do arquivo xlsx usando pandas
df = pd.read_excel(file_path, sheet_name='BDados')

# Inicializa variáveis necessárias
dataResp = {}
count = 0
timeAnterior = []  # Inicializa timeAnterior como uma lista vazia

# Loop para processar colunas que começam com "Tela"
for indexCol, colStr in enumerate(df.columns):

    if colStr.startswith("Tela") and df.iloc[0, indexCol] != "" and pd.notna(df.iloc[0, indexCol]):
        timeAtual = []
        timeElipsed = []
        value = []
        
        colNum = int(re.search(r"Tela (\d+)", colStr).group(1))
        
        for indexRow in range(df.shape[0]):
            cell_value = str(df.iloc[indexRow, indexCol])
            match = re.match(r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}) - (.+)", cell_value)
            
            if match:
                timeAtual.append(pd.to_datetime(match.group(1)))
                if colNum != 1:                                    
                    trataCol(dataResp, colNum, match.group(2), timeAtual[-1], timeAnterior[indexRow]) # globals()[f"func_tela_colNum"](match.group(2))
            else:
                print("Ocorreu um erro ao processar a célula:", cell_value)
        
        timeAnterior = timeAtual 
        
# Cria o novo DataFrame com os dados processados
df_novo = pd.DataFrame(dataResp)

# Usando ExcelWriter para adicionar a nova aba ao arquivo Excel existente com if_sheet_exists='replace'
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_novo.to_excel(writer, sheet_name='TDados', index=False)